** 模型预测 **

In [4]:
import os
from keras.preprocessing import image
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping,ModelCheckpoint
from PIL import Image
from keras import optimizers
from keras.applications import *
from keras.models import *
from keras.layers import *
from bottleneck_ir2 import create_inception_resnet_v2

In [ ]:
#初始化模型
def model(MODEL,width=299,height=299,lambda_func=None):
    x= Input((height, width, 3))
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model=MODEL(input_tensor=x, weights='imagenet', include_top=False)
    return Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

#格式化输入
def data_input(imgpath,width=299,height=299):
    img= image.load_img(path=imgpath, target_size=(height, width))
    return np.expand_dims(image.img_to_array(img), axis=0)
#预训练模型的输入
def input_n(imgpath,model_name,width=299,height=299,lambda_func=None):
    format_input=data_input(imgpath,width,height)
    m=model(model_name,width,height,lambda_func)
    return m.predict(format_input)

imgpath='1.jpg'
input1=input_n(imgpath,ResNet50,224,224)
input2=input_n(imgpath,Xception,299,299,xception.preprocess_input)

m=create_inception_resnet_v2()
m.load_weights('weight-v1.h5',by_name=True)
m=Model(m.input, GlobalAveragePooling2D()(m.output))
img= image.load_img(path=imgpath, target_size=(299, 299))
a=np.expand_dims(np.asarray(img,dtype="float32")/255, axis=0)
input3=m.predict(a)

x_input=[]
x_input.append(np.array(input1))
x_input.append(np.array(input2))
x_input.append(np.array(input3))
x_input = np.concatenate(x_input, axis=1)


from keras.models import load_model
model=load_model('model_bottleneck.h5')
pred=model.predict(x_input, verbose=1)
pred = pred.clip(min=0.005, max=0.995)
print(pred)
